In [3]:
!pip install  neo4j
!pip install langchain-groq

   ---------------------------------------- 0.0/103.5 kB ? eta -:--:--
   ---------------------------------------- 103.5/103.5 kB 3.0 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv
load_dotenv()
import os

In [3]:
groq_api_key=os.getenv("GROQ_API_KEY")
NEO4J_URI=os.getenv("NEO4J_URI")
NEO4J_USERNAME=os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD=os.getenv("NEO4J_PASSWORD")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

In [4]:
os.environ['NEO4J_URI']=NEO4J_URI
os.environ['NEO4J_USER']=NEO4J_USERNAME
os.environ['NEO4J_PASSWORD']=NEO4J_PASSWORD

In [5]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [38]:
graph

In [39]:
!pip install langchain-google-genai

In [40]:
from langchain_groq import ChatGroq

llm_groq=ChatGroq(
    groq_api_key=groq_api_key,
    model_name="gemma-7b-it"
)
llm_groq

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002C7B59154F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002C7B5BFBA60>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [41]:
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="models/gemini-pro", google_api_key=GOOGLE_API_KEY)

In [58]:
!pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11704 sha256=3be18162ce5ca032d2af3a1b27365ffc4ff82c5fa38169bdd8bc1ecfe326a2e0
  Stored in directory: c:\users\ayush\appdata\local\pip\cache\wheels\c2\46\f4\caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia


In [65]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Elizabeth I").load()
raw_documents

a:\LANGCHAIN\langchainenv\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file a:\LANGCHAIN\langchainenv\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


[Document(metadata={'title': 'Elizabeth I', 'summary': 'Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last monarch of the House of Tudor.\nElizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents\' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry\'s death in 1547, Elizabeth\'s younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward\'s will was set aside within weeks of his death and Mary became queen, deposing and executing Jane. During Mary\'s reign, Elizabeth was i

In [66]:
print(len(raw_documents))
print(raw_documents[0])

23
page_content='Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last monarch of the House of Tudor.
Elizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry's death in 1547, Elizabeth's younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward's will was set aside within weeks of his death and Mary became queen, deposing and executing Jane. During Mary's reign, Elizabeth was imprisoned for nearly a year on suspicion of s

In [67]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents)

In [68]:
len(documents)

46

In [ ]:
!pip install langchain_experimental

In [69]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm_groq)

In [71]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [51]:
graph_documents[0].nodes


[Node(id='Narendra Damodardas Modi', type='Person'),
 Node(id='Vadnagar', type='Location'),
 Node(id='Gujarat', type='Location'),
 Node(id='India', type='Country'),
 Node(id='Bharatiya Janata Party', type='Organisation'),
 Node(id='Rashtriya Swayamsevak Sangh', type='Organisation'),
 Node(id='Prime Minister Of India', type='Role'),
 Node(id='Chief Minister Of Gujarat', type='Role'),
 Node(id='Member Of Parliament', type='Role'),
 Node(id='Varanasi', type='Location'),
 Node(id='Jashodaben Modi', type='Person'),
 Node(id='Supreme Court Of India', type='Organisation')]

In [53]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [64]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Document {id: STRING, text: STRING}
Person {id: STRING}
Organization {id: STRING}
Government_body {id: STRING}
Location {id: STRING}
Relationship properties:

The relationships:
(:Document)-[:MENTIONS]->(:Person)
(:Document)-[:MENTIONS]->(:Organization)
(:Document)-[:MENTIONS]->(:Government_body)
(:Document)-[:MENTIONS]->(:Location)
(:Person)-[:SPOUSE]->(:Person)
(:Person)-[:MEMBER]->(:Organization)
(:Person)-[:MEMBER]->(:Government_body)
(:Person)-[:CHIEF_MINISTER]->(:Location)
(:Person)-[:BORN]->(:Location)
(:Person)-[:REPRESENTS]->(:Location)
(:Person)-[:RESIDES]->(:Location)
(:Person)-[:PRIME_MINISTER]->(:Location)


In [56]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm_groq,graph=graph,verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x000002C7B5A1A070>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002C7B59154F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002C7B5BFBA60>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [64]:
chain.invoke({"query":" prime minister of india?"})



> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: India)} {position: line: 1, column: 43, offset: 42} for query: 'MATCH (p:Person)<-[:PRIME_MINISTER_OF]-(i:India) RETURN p.name \n'


Generated Cypher:
MATCH (p:Person)<-[:PRIME_MINISTER_OF]-(i:India) RETURN p.name 



Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: name)} {position: line: 1, column: 59, offset: 58} for query: 'MATCH (p:Person)<-[:PRIME_MINISTER_OF]-(i:India) RETURN p.name \n'


Full Context:
[]

> Finished chain.


{'query': ' prime minister of india?', 'result': "I don't know the answer. \n"}